In [1]:
using Crystalline, MPBUtils
using PyCall
mp = pyimport("meep")
mpb = pyimport("meep.mpb")

┌ Warning: PyNormaliz could not be imported: related functionality of SymmetryBases.jl is nonfunctional
└ @ SymmetryBases /Users/antoniomoralesperez/.julia/packages/SymmetryBases/fpjO6/src/SymmetryBases.jl:26


PyObject <module 'meep.mpb' from '/opt/homebrew/Caskroom/miniconda/base/envs/mpb/lib/python3.12/site-packages/meep/mpb/__init__.py'>

### Example on SG211

Let me try to make an specific example but using the Crystalline package and it functionalities instead of doing it manually

#### Implementing my code in Julia

Things we need:

1. Symmetry vector from MPB
2. Irreps, EBRs and dimensions of EBRs of the SG 

In [2]:
# point 1
R1 = 0.2 #cylinder radius
mat = mp.Medium(epsilon=12)
geometry = [
    mp.Cylinder(radius=R1, center=[0, 0, 0], axis=[0, 0, 1], height=1, material=mat),
    mp.Cylinder(radius=R1, center=[0, 0, 0], axis=[0, 1, 0], height=1, material=mat),
    mp.Cylinder(radius=R1, center=[0, 0, 0], axis=[1, 0, 0], height=1, material=mat),
]
ms = mpb.ModeSolver(
    num_bands=8,
    geometry_lattice=mp.Lattice(basis1=[1, 0, 0], basis2=[0, 1, 0], basis3=[0, 0, 1],
        size=[1, 1, 1]),
    geometry=geometry,
    resolution=32
)
ms.init_params(p=mp.ALL, reset_fields=true)

# point 2
sg_num = 221                                        # P-4 (Z₂×Z₂ symmetry indicator group)
brs = bandreps(sg_num)                             # elementary band representations
lgs = littlegroups(sg_num)                         # little groups
filter!(((klab, _),) -> klab ∈ klabels(brs), lgs) # restrict to k-points in `brs`
map!(lg -> primitivize(lg, false), values(lgs))   # convert to primitive setting (without reducing translations)
lgirsd = pick_lgirreps(lgs; timereversal=true);    # small irreps associated with `lgs`

symeigsd = Dict{String,Vector{Vector{ComplexF64}}}()
for (klab, lg) in lgs
    kv = mp.Vector3(position(lg)()...)
    ms.solve_kpoint(kv)

    symeigsd[klab] = [Vector{ComplexF64}(undef, length(lg)) for n in 1:ms.num_bands]
    for (i, gᵢ) in enumerate(lg)
        W = mp.Matrix(eachcol(rotation(gᵢ))...) # decompose gᵢ = {W|w}
        w = mp.Vector3(translation(gᵢ)...)
        symeigs = ms.compute_symmetries(W, w)   # compute ⟨Eₙₖ|gᵢDₙₖ⟩ for all bands
        setindex!.(symeigsd[klab], symeigs, i)  # update container of symmetry eigenvalues
    end
end

Working in 3 dimensions.
Grid size is 32 x 32 x 32.
Solving for 8 bands at a time.
Creating Maxwell data...
Mesh size is 3.
Lattice vectors:
     (1, 0, 0)
     (0, 1, 0)
     (0, 0, 1)
Cell volume = 1
Reciprocal lattice vectors (/ 2 pi):
     (1, -0, 0)
     (-0, 1, -0)
     (0, -0, 1)
Geometric objects:
     cylinder, center = (0,0,0)
          radius 0.2, height 1, axis (0, 0, 1)
     cylinder, center = (0,0,0)
          radius 0.2, height 1, axis (0, 1, 0)
     cylinder, center = (0,0,0)
          radius 0.2, height 1, axis (1, 0, 0)
Geometric object tree has depth 7 and 15 object nodes (vs. 3 actual objects)
Initializing epsilon function...
Allocating fields...
Solving for band polarization: .
Initializing fields to random numbers...
solve_kpoint (0.5,0.5,0):
freqs:, k index, k1, k2, k3, kmag/2pi, band 1, band 2, band 3, band 4, band 5, band 6, band 7, band 8
Solving for bands 1 to 8...
    near maximum in trace
    linmin: converged after 7 iterations.
    iteration    1: trace =

In [3]:
# --- fix singular photonic symmetry content at Γ, ω=0 ---
fixup_gamma_symmetry!(symeigsd, lgs)

# --- analyze connectivity and topology of symmetry data ---
summaries = analyze_symmetry_data(symeigsd, lgirsd, brs)

2-element Vector{BandSummary}:
 2-band (fragile): [-Γ₁⁺+Γ₄⁻, R₃⁺, M₂⁺+M₃⁻, X₅⁻]
 3-band (trivial): [Γ₄⁺, R₄⁺, M₃⁺+M₅⁺, X₃⁺+X₅⁺]

In [25]:
B = matrix(brs) # matrix of EBRs with their dimensions in the last columns
d = B[end, :]


40-element Vector{Int64}:
 1
 1
 1
 1
 2
 2
 3
 3
 3
 3
 ⋮
 3
 3
 3
 3
 3
 3
 3
 6
 6

#### First we need to recover all information

1. Space group generators, irreps, WPs and EBRs
2. Cosets decompositions of WPs
3. Site-symmetry group representations of WPs
4. $n$-nearest neighbors positions and distances
5. 

In [1]:

# point 1
sg_num = 221;
sg_syms = spacegroup(sg_num); # list of all symmetries in the SG
gens = generators(sg_num); # list of the generators of the SG
lgirs = lgirreps(sg_num); # dictionary of all little group irreps
wps = wyckoffs(sg_num); # list of all WPs in the SG
brs = bandreps(sg_num); # list of all elementary band representations in the SG

# now we need to fix a WP and probably an EBR or just a site-symmetry group
# let me pick A_2g@3c
fix_wp = wp[end-2]
fix_ebr = brs[23] # TODO: I had to do this by hand. Maybe there is a way to do it automatically

# point 2
sitesg = sitegroup(sg_num, fix_wp); # site-symmetry group of the fixed WP
coset_decomp = cosets(sitesg); # coset representatives of the sitesg

# point 3
point_representation = siteirreps(sitesg);

UndefVarError: UndefVarError: `spacegroup` not defined

In [30]:
sitesg

SiteGroup{3} ⋕221 (Pm-3m) at 3c = [0, 1/2, 1/2] with 16 operations:
 1
 {2₀₀₁|0,1,0}
 {2₀₁₀|0,0,1}
 {2₁₀₀|0,1,1}
 {4₁₀₀⁻|0,0,1}
 2₀₁₁
 {2₀₋₁₁|0,1,1}
 {4₁₀₀⁺|0,1,0}
 {-1|0,1,1}
 {m₀₀₁|0,0,1}
 {m₀₁₀|0,1,0}
 m₁₀₀
 {-4₁₀₀⁻|0,1,0}
 {m₀₁₁|0,1,1}
 m₀₋₁₁
 {-4₁₀₀⁺|0,0,1}